In [1]:
import pandas as pd 
from dateutil import parser

In [2]:
df = pd.read_csv("../PJME_hourly.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values(by = 'Datetime')
df.reset_index(inplace = True , drop = True)
df.head()

Datetime  PJME_MW
0 2002-01-01 01:00:00  30393.0
1 2002-01-01 02:00:00  29265.0
2 2002-01-01 03:00:00  28357.0
3 2002-01-01 04:00:00  27899.0
4 2002-01-01 05:00:00  28057.0

In [3]:
class FeatureEngineering:
    DAYS_IN_MONTH = [31 , 28 , 31 , 30 , 31 , 30 , 31 , 31 , 30 , 31 , 30 , 31]
    
    def getProperDay(self,month ):
        return  self.DAYS_IN_MONTH[month-1]
    
    
    def getLastMonthHelper(self , value , planB):
         try:
            value = parser.parse(value)
        except: 
            planB = parser.parse(planB)
        
        if type(value) == str :
            value = planB

        ans = df[df['Datetime'] == value]['PJME_MW']
        return ans 
    
    def getLastMonth(self , date  , df ):
        day = date.day
        month = date.month
        year = date.year
        hour = date.hour
        if month == 1:
            month = 13
            year -= 1
            
        value = f"{month-1}-{day}-{year} {hour}:00:00"
        planB = f"{month-1}-{self.getProperDay(month-1)}-{year} {hour}:00:00"

        ans =  self.getLastMonthHelper(value , planB)
        if ans.any():
            return ans.iloc[0]
        
        return df[df['Datetime'] == date]['PJME_MW'].iloc[0]
    

In [4]:
def handleMissings(lastWeek , sameValue):
    
    if str(lastWeek) == 'nan':
        return sameValue    
    return lastWeek

FE = FeatureEngineering()
df['lastMonth'] = df['Datetime'].apply(lambda x : FE.getLastMonth(x , df))
df['lastWeek'] = df['PJME_MW'].shift(168)
df['lastWeek'] = df.apply(lambda x : handleMissings(x['lastWeek'] , x['PJME_MW']) , axis = 1 )

### Validate Feature Engineering Phase

In [5]:
df.head()

Datetime  PJME_MW  lastMonth  lastWeek
0 2002-01-01 01:00:00  30393.0    30393.0   30393.0
1 2002-01-01 02:00:00  29265.0    29265.0   29265.0
2 2002-01-01 03:00:00  28357.0    28357.0   28357.0
3 2002-01-01 04:00:00  27899.0    27899.0   27899.0
4 2002-01-01 05:00:00  28057.0    28057.0   28057.0

In [15]:
#check if everythings works well 
testValue = str(df.Datetime.sample(1).iloc[0])
testValue

'2008-03-12 06:00:00'

In [16]:
df[df['Datetime']==testValue]

Datetime  PJME_MW  lastMonth  lastWeek
54280 2008-03-12 06:00:00  29219.0    36706.0   24565.0

In [26]:
df[ (df['Datetime'] == '2008-02-12 06:00:00')].iloc[0] , df.iloc[54280-168]

(Datetime     2008-02-12 06:00:00
 PJME_MW                  36706.0
 lastMonth                26247.0
 lastWeek                 29565.0
 Name: 53585, dtype: object,
 Datetime     2008-03-05 05:00:00
 PJME_MW                  24565.0
 lastMonth                27368.0
 lastWeek                 28409.0
 Name: 54112, dtype: object)

### Great, Seems Like everything's works well 

In [27]:
df.to_csv("energy.csv" , index = False)

In [28]:
pd.read_csv("energy.csv")

Datetime  PJME_MW  lastMonth  lastWeek
0       2002-01-01 01:00:00  30393.0    30393.0   30393.0
1       2002-01-01 02:00:00  29265.0    29265.0   29265.0
2       2002-01-01 03:00:00  28357.0    28357.0   28357.0
3       2002-01-01 04:00:00  27899.0    27899.0   27899.0
4       2002-01-01 05:00:00  28057.0    28057.0   28057.0
...                     ...      ...        ...       ...
145361  2018-08-02 20:00:00  44057.0    54644.0   46337.0
145362  2018-08-02 21:00:00  43256.0    52757.0   44542.0
145363  2018-08-02 22:00:00  41552.0    50685.0   42638.0
145364  2018-08-02 23:00:00  38500.0    46833.0   39276.0
145365  2018-08-03 00:00:00  35486.0    42919.0   35742.0

[145366 rows x 4 columns]